In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset



c:\Users\rsaxe\.conda\envs\yolov8\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Example data for demonstration
time_steps = np.arange(1, 101)
x1 = np.sin(time_steps * 0.1) + np.random.normal(0, 0.05, 100)
x2 = np.cos(time_steps * 0.1) + np.random.normal(0, 0.05, 100)
x3 = np.sin(time_steps * 0.05) + np.random.normal(0, 0.05, 100)
x4 = np.cos(time_steps * 0.05) + np.random.normal(0, 0.05, 100)
y = np.sin(time_steps * 0.1) + np.cos(time_steps * 0.1) + np.random.normal(0, 0.1, 100)

# Combine into a dataframe
data = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'y': y})

# Display first few rows
print(data.head())


         x1        x2        x3        x4         y
0  0.156083  0.944126  0.031287  0.933191  1.107611
1  0.261743  1.008343  0.071159  1.029584  1.220628
2  0.241827  0.999943  0.103662  1.006220  1.269037
3  0.388077  0.879201  0.193663  0.941076  1.156001
4  0.494463  0.874472  0.313309  1.048023  1.173761


In [3]:
def create_sequences_multivariate(data, seq_length):
    sequences = []
    labels = []
    
    for i in range(len(data) - seq_length):
        # Extract sequences of features (x1, x2, x3, x4) for `seq_length` time steps
        seq = data.iloc[i:i+seq_length, :-1].values  # All columns except target 'y'
        label = data.iloc[i+seq_length, -1]  # The target column 'y'
        
        sequences.append(seq)
        labels.append(label)

    return np.array(sequences), np.array(labels)


In [4]:
# Define sequence length
seq_length = 10

# Create sequences and labels
X, y = create_sequences_multivariate(data, seq_length)

print("Shape of X:", X.shape)  # Expected: (num_samples, seq_length, num_features)
print("Shape of y:", y.shape)  # Expected: (num_samples,)


Shape of X: (90, 10, 4)
Shape of y: (90,)


In [8]:


# Convert to PyTorch tensors
X_tensor = torch.FloatTensor(X)
y_tensor = torch.FloatTensor(y)

print("Shape of X_tensor:", X_tensor.shape)  # (num_samples, seq_length, num_features)
print("Shape of y_tensor:", y_tensor.shape)  # (num_samples,)


Shape of X_tensor: torch.Size([90, 10, 4])
Shape of y_tensor: torch.Size([90])


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")


Training samples: 72
Testing samples: 18


In [10]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=4, hidden_size=64, num_layers=2, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # LSTM forward pass
        out, _ = self.lstm(x, (h0, c0))
        
        # Output from last time step
        out = self.fc(out[:, -1, :])  
        return out
